# WideDeep_BayesianWide

## 라이브러리 설치

In [1]:
# !pip install numpy==1.26.0
# !pip install pandas==2.2.2
# !pip install scikit-learn==1.5.1
# !pip install scipy==1.14.1
# !pip install statsmodels==0.14.2
# !pip install joblib==1.4.2
# !pip install threadpoolctl==3.5.0
# !pip install ipynbname

In [2]:
# # 1. 기존의 패키지 정리
# !pip uninstall -y torch torchvision torchaudio pytorch-lightning pytorch-tabular

# # 2. 호환 가능한 버전으로 재설치
# !pip install torch==2.0.1+cu118 --index-url https://download.pytorch.org/whl/cu118
# !pip install pytorch-tabular==1.1.1 --no-deps
# !pip install pytorch-lightning==2.0.0

In [3]:
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [4]:
# !pip install pytorch-widedeep

In [5]:
# !pip install opencv-python-headless

## import

In [6]:
import pandas as pd
import numpy as np
import sqlite3
import shutil
import ipynbname
import datetime
import os
import warnings
import random
warnings.simplefilter(action='ignore', category=FutureWarning)


from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder, FunctionTransformer, QuantileTransformer, MultiLabelBinarizer
from sklearn.impute import SimpleImputer



import torch

# pytorch-widedeep 라이브러리 import
from pytorch_widedeep import Trainer
from pytorch_widedeep.preprocessing import WidePreprocessor, TabPreprocessor
from pytorch_widedeep.models import Wide, FTTransformer, WideDeep
from pytorch_widedeep.bayesian_models import BayesianWide, BayesianTabMlp
from pytorch_widedeep.training.bayesian_trainer import BayesianTrainer
from pytorch_widedeep.metrics import Accuracy,F1Score
from pytorch_widedeep.callbacks import EarlyStopping, ModelCheckpoint


##################
data_seed=1

train_path = f'./data/custom_train_{data_seed}.csv'
test_path = f'./data/custom_test_{data_seed}.csv'


train_path = f'C:/Users/User/Desktop/LG Aimers/LG_Aimers_6th/data/custom_train_{data_seed}.csv'
test_path = f'C:/Users/User/Desktop/LG Aimers/LG_Aimers_6th/data/custom_test_{data_seed}.csv'

## Data Load

In [7]:
train = pd.read_csv(train_path).drop(columns=["ID"])
test = pd.read_csv(test_path).drop(columns=["ID"])
print(train.shape, test.shape)

(205080, 68) (51271, 67)


## Data Pre-processing

In [8]:
def drop_columns(df):
    cols = [
        '불임 원인 - 여성 요인',  # 고유값 1
        '불임 원인 - 정자 면역학적 요인',  # train, test 모두 '1'인 데이터 1개 >> 신뢰할 수 없음
        '난자 해동 경과일',
    ]
    df = df.drop(cols, axis=1)
    return df

def 특정시술유형(train, test):
    def categorize_procedure(proc):
        tokens = [token.strip() for token in proc.split(",") if token.strip() and not token.strip().isdigit()]
        # 우선순위에 따른 범주화
        if tokens.count("Unknown") >= 1:
            return "Unknown"
        if tokens.count("AH") >= 1:
            return "AH"
        if tokens.count("BLASTOCYST") >= 1:
            return "BLASTOCYST"
        if tokens.count("ICSI") >= 2 or tokens.count("IVF") >= 2:
            return "2ICSI_2IVF"
        if tokens.count("IVF") >= 1 and tokens.count("ICSI") >= 1:
            return "IVF_ICSI"
        if tokens == "ICSI":
            return "ICSI"
        if tokens == "IVF":
            return "IVF"
        return ",".join(tokens) if tokens else None

    for df in [train, test]:
        df['특정 시술 유형'] = df['특정 시술 유형'].str.replace(" / ", ",")
        df['특정 시술 유형'] = df['특정 시술 유형'].str.replace(":", ",")
        df['특정 시술 유형'] = df['특정 시술 유형'].str.replace(" ", "")

    counts = train['특정 시술 유형'].value_counts()
    allowed_categories = counts[counts >= 100].index.tolist()

    # allowed_categories에 속하지 않는 값은 "Unknown"으로 대체
    train.loc[~train['특정 시술 유형'].isin(allowed_categories), '특정 시술 유형'] = "Unknown"
    test.loc[~test['특정 시술 유형'].isin(allowed_categories), '특정 시술 유형'] = "Unknown"

    train['특정 시술 유형'] = train['특정 시술 유형'].apply(categorize_procedure)
    test['특정 시술 유형'] = test['특정 시술 유형'].apply(categorize_procedure)

    train['시술유형_통합'] = train['시술 유형'].astype(str) + '_' + train['특정 시술 유형'].astype(str)
    test['시술유형_통합'] = test['시술 유형'].astype(str) + '_' + test['특정 시술 유형'].astype(str)

    drop_cols = ['시술 유형', '특정 시술 유형']
    train = train.drop(drop_cols, axis=1)
    test = test.drop(drop_cols, axis=1)

    return train, test

def 시술횟수(df_train):
    for col in [col for col in df_train.columns if '횟수' in col]:
        df_train[col] = df_train[col].replace({'6회 이상':'6회'})
        df_train[col] = df_train[col].str[0].astype(int)
    df_train['시술_임신'] = df_train['총 임신 횟수'] - df_train['총 시술 횟수']
    df_train = df_train.drop('총 시술 횟수', axis=1)
    return df_train

def 배란유도유형(df_train, df_test):
    mapping = {
        '기록되지 않은 시행': 1,
        '알 수 없음': 0,
        '세트로타이드 (억제제)': 0,
        '생식선 자극 호르몬': 0,
    }
    df_train['배란 유도 유형'] = df_train['배란 유도 유형'].replace(mapping)
    df_test['배란 유도 유형'] = df_test['배란 유도 유형'].replace(mapping)
    return df_train, df_test

def 난자기증자나이(df_train, df_test):
    mapping = {
        '만20세 이하': 20,
        '만21-25세': 25,
        '만26-30세': 30,
        '만31-35세': 35,
        '알 수 없음': 20,  # 만20세 이하와 동일하게 처리
    }
    df_train['난자 기증자 나이'] = df_train['난자 기증자 나이'].replace(mapping)
    df_test['난자 기증자 나이'] = df_test['난자 기증자 나이'].replace(mapping)
    return df_train, df_test

def 배아생성주요이유(df_train, df_test):
    df_train['배아 생성 주요 이유'] = df_train['배아 생성 주요 이유'].fillna('DI')
    df_test['배아 생성 주요 이유'] = df_test['배아 생성 주요 이유'].fillna('DI')

    df_train['배아 생성 이유 리스트'] = df_train['배아 생성 주요 이유'].apply(lambda x: [reason.strip() for reason in x.split(',')])
    df_test['배아 생성 이유 리스트'] = df_test['배아 생성 주요 이유'].apply(lambda x: [reason.strip() for reason in x.split(',')])

    mlb = MultiLabelBinarizer()
    train_one_hot = pd.DataFrame(
        mlb.fit_transform(df_train['배아 생성 이유 리스트']),
        columns=mlb.classes_,
        index=df_train.index
    )
    train_one_hot.columns = ['배아생성이유_' + col for col in train_one_hot.columns]

    test_one_hot = pd.DataFrame(
        mlb.transform(df_test['배아 생성 이유 리스트']),
        columns=mlb.classes_,
        index=df_test.index
    )
    test_one_hot.columns = ['배아생성이유_' + col for col in test_one_hot.columns]

    df_train = pd.concat([df_train, train_one_hot], axis=1)
    df_test = pd.concat([df_test, test_one_hot], axis=1)

    cols_to_drop = [
        '배아 생성 주요 이유',
        '배아 생성 이유 리스트',
        '배아생성이유_연구용',
        '배아생성이유_DI'
    ]
    df_train = df_train.drop(cols_to_drop, axis=1, errors='ignore')
    df_test = df_test.drop(cols_to_drop, axis=1, errors='ignore')

    cols = ['배아생성이유_기증용',
            '배아생성이유_난자 저장용',
            '배아생성이유_배아 저장용',
            '배아생성이유_현재 시술용']

    df_train[cols] = df_train[cols].div(df_train[cols].sum(axis=1).replace(0, np.nan), axis=0).fillna(0)
    df_test[cols] = df_test[cols].div(df_test[cols].sum(axis=1).replace(0, np.nan), axis=0).fillna(0)

    return df_train, df_test

def 단일배아이식여부(df_train, df_val):
    df_train['단일 배아 이식 여부'] = df_train['단일 배아 이식 여부'].fillna(0)
    df_val['단일 배아 이식 여부'] = df_val['단일 배아 이식 여부'].fillna(0)
    return df_train, df_val


def 기증자정자와혼합된난자수(df_train, df_test):
    df_train["기증자 정자와 혼합된 난자 수"] = df_train["기증자 정자와 혼합된 난자 수"].fillna(2)
    df_test["기증자 정자와 혼합된 난자 수"] = df_test["기증자 정자와 혼합된 난자 수"].fillna(2)
    return df_train, df_test

def label_encoding(train, test, cols):
    encoder = LabelEncoder()
    for col in cols:
        train[col] = encoder.fit_transform(train[col])
        test[col] = encoder.transform(test[col])
    return train, test

def type_to_category(train, test, cols):
    train[cols] = train[cols].astype('category')
    test[cols] = test[cols].astype('category')
    return train, test

def impute_nan(train, test):
    cols_to_impute = [
        '임신 시도 또는 마지막 임신 경과 연수', # DI, IVF랑 관련 X
    ]
    imputer = SimpleImputer(strategy='mean')
    train[cols_to_impute] = imputer.fit_transform(train[cols_to_impute])
    test[cols_to_impute] = imputer.transform(test[cols_to_impute])

    cols_to_impute = [
        '난자 채취 경과일',
        '난자 혼합 경과일',
        '배아 이식 경과일',
        '배아 해동 경과일',

        '착상 전 유전 검사 사용 여부',
        'PGD 시술 여부',
        'PGS 시술 여부',

        ### DI only
        '착상 전 유전 진단 사용 여부',
        '총 생성 배아 수',
        '미세주입된 난자 수',
        '미세주입에서 생성된 배아 수',
        '이식된 배아 수',
        '미세주입 배아 이식 수',
        '저장된 배아 수',
        '미세주입 후 저장된 배아 수',
        '해동된 배아 수',
        '해동 난자 수',
        '수집된 신선 난자 수',
        '저장된 신선 난자 수',
        '혼합된 난자 수',
        '파트너 정자와 혼합된 난자 수',
        '기증자 정자와 혼합된 난자 수',
        '동결 배아 사용 여부',
        '신선 배아 사용 여부',
        '기증 배아 사용 여부',
        '대리모 여부',
        ### DI
    ]
    train[cols_to_impute] = train[cols_to_impute].fillna(0)
    test[cols_to_impute] = test[cols_to_impute].fillna(0)

    return train, test

def num_feature_scailing(train, test, seed=777):
    numeric_cols = train.select_dtypes(include=["number"]).columns.tolist()
    cat_cols = [col for col in train.columns if pd.api.types.is_categorical_dtype(train[col])]
    cols_to_scale = [
        col for col in numeric_cols
        if col not in cat_cols and col != '임신 성공 여부'
    ]

    arr_train = train[cols_to_scale].to_numpy()  # DataFrame -> NumPy
    arr_train = arr_train.astype(np.float32)
    arr_test = test[cols_to_scale].to_numpy()
    arr_test = arr_test.astype(np.float32)

    np.random.seed(seed)
    random.seed(seed)
    noise = (
        np.random.default_rng(0)
        .normal(0.0, 1e-5, arr_train.shape)
        .astype(arr_train.dtype)
    )
    preprocessing = QuantileTransformer(
        n_quantiles=max(min(len(train[cols_to_scale]) // 30, 1000), 10),
        output_distribution='normal',
        subsample=10**9,
    ).fit(arr_train + noise)

    # train[cols_to_scale] = preprocessing.transform(arr_train + noise)
    train[cols_to_scale] = preprocessing.transform(arr_train)
    test[cols_to_scale] = preprocessing.transform(arr_test)
    return train, test

def drop_single_value_columns(df_train, df_test):
    cols_to_drop = [col for col in df_train.columns if df_train[col].nunique() == 1]
    return df_train.drop(columns=cols_to_drop), df_test.drop(columns=cols_to_drop)





In [9]:
def all_process(train, val):
    # 기본 전처리 단계
    train, val = drop_columns(train), drop_columns(val)
    train, val = 특정시술유형(train, val)
    train, val = 시술횟수(train), 시술횟수(val)

    train, val = 단일배아이식여부(train, val)
    train, val = 배란유도유형(train, val)
    train, val = 배아생성주요이유(train, val)

    cols_to_encoding = [
        "시술 시기 코드",
        "시술 당시 나이",
        "배란 유도 유형",
        # "클리닉 내 총 시술 횟수",
        # "IVF 시술 횟수",
        # "DI 시술 횟수",
        # "총 임신 횟수",
        # "IVF 임신 횟수",
        # "DI 임신 횟수",
        # "총 출산 횟수",
        # "IVF 출산 횟수",
        # "DI 출산 횟수",
        "난자 출처",
        "정자 출처",
        "난자 기증자 나이",
        "정자 기증자 나이",
        '시술유형_통합',
    ]
    train, val = label_encoding(train, val, cols=cols_to_encoding)
    train, val = type_to_category(train, val, cols=cols_to_encoding)

    train, val = impute_nan(train, val)
    train, val = num_feature_scailing(train, val)

    train, val = drop_single_value_columns(train, val)

    return train, val

train = pd.read_csv(train_path).drop(columns=["ID"])
test = pd.read_csv(test_path).drop(columns=["ID"])

train, test = all_process(train, test)

cat_cols = [col for col in train.columns if pd.api.types.is_categorical_dtype(train[col])]
numeric_cols = [col for col in train.columns if col not in cat_cols and col != '임신 성공 여부']

print(f'수치형 변수: {len(numeric_cols)}개 \n{numeric_cols}')
print(f'범주형 변수: {len(cat_cols)}개 \n{cat_cols}')
print(train.shape, test.shape)

C:\Users\User\AppData\Local\Temp\ipykernel_5824\3576064510.py:69: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_train['배란 유도 유형'] = df_train['배란 유도 유형'].replace(mapping)
C:\Users\User\AppData\Local\Temp\ipykernel_5824\3576064510.py:70: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_test['배란 유도 유형'] = df_test['배란 유도 유형'].replace(mapping)
C:\Users\User\AppData\Local\Temp\ipykernel_5824\3576064510.py:199: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.Cate

수치형 변수: 57개 
['임신 시도 또는 마지막 임신 경과 연수', '배란 자극 여부', '단일 배아 이식 여부', '착상 전 유전 검사 사용 여부', '착상 전 유전 진단 사용 여부', '남성 주 불임 원인', '남성 부 불임 원인', '여성 주 불임 원인', '여성 부 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인', '불명확 불임 원인', '불임 원인 - 난관 질환', '불임 원인 - 남성 요인', '불임 원인 - 배란 장애', '불임 원인 - 자궁경부 문제', '불임 원인 - 자궁내막증', '불임 원인 - 정자 농도', '불임 원인 - 정자 운동성', '불임 원인 - 정자 형태', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', '총 임신 횟수', 'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수', '총 생성 배아 수', '미세주입된 난자 수', '미세주입에서 생성된 배아 수', '이식된 배아 수', '미세주입 배아 이식 수', '저장된 배아 수', '미세주입 후 저장된 배아 수', '해동된 배아 수', '해동 난자 수', '수집된 신선 난자 수', '저장된 신선 난자 수', '혼합된 난자 수', '파트너 정자와 혼합된 난자 수', '기증자 정자와 혼합된 난자 수', '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부', '대리모 여부', 'PGD 시술 여부', 'PGS 시술 여부', '난자 혼합 경과일', '배아 이식 경과일', '배아 해동 경과일', '시술_임신', '배아생성이유_기증용', '배아생성이유_난자 저장용', '배아생성이유_배아 저장용', '배아생성이유_현재 시술용']
범주형 변수: 8개 
['시술 시기 코드', '시술 당시 나이', '배란 유도 유형', '난자 출처', '정자 출처', '난자 기증자 나이', '정자 기증자 나이', '시술유형_통합']
(205080, 66) (51271, 65)


C:\Users\User\AppData\Local\Temp\ipykernel_5824\2392652077.py:45: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  cat_cols = [col for col in train.columns if pd.api.types.is_categorical_dtype(train[col])]


In [10]:
# 실험 내용
experiment_desc = '''
WideDeep_BayesianWide 임신 성공 여부 

'''

In [11]:
import time
start_time = time.time()


# import scipy.special   # 시그모이드 대신 softmax에 사용할 수 있음
# 교차 검증 설정: seed_list를 [333] 하나만 사용, n_splits=3
seed_list = [333]
n_splits = 5

total_auc, total_acc, total_f1 = [], [], []
test_preds = []





# 교차 검증 시작
for seed in seed_list:
    # train, test 불러오기
    train = pd.read_csv(train_path).drop(columns=["ID"])
    test = pd.read_csv(test_path).drop(columns=["ID"])

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    auc_scores, acc_scores, f1_scores = [], [], []
    
    for fold, (train_idx, valid_idx) in enumerate(skf.split(train.drop(columns=['임신 성공 여부']), train["임신 성공 여부"])):
        # Fold 데이터 생성
        fold_train, fold_valid = train.iloc[train_idx], train.iloc[valid_idx]
        fold_train2 = fold_train.copy()
        fold_test = test.copy()  # test 데이터는 별도 사용
        
        # 전처리 
        fold_train, fold_valid = all_process(fold_train, fold_valid)
        _, fold_test = all_process(fold_train2, fold_test)

        # 범주형, 연속형 열 구분분
        categorical_cols = [col for col in fold_train.columns if pd.api.types.is_categorical_dtype(fold_train[col])]
        continuous_cols = [col for col in fold_train.columns if col not in cat_cols and col != '임신 성공 여부']



        # Deep 부분: 임베딩 + 연속형 변수 처리
        tab_preprocessor = TabPreprocessor(embed_cols=categorical_cols, continuous_cols=continuous_cols)

        # 전처리: 각 Fold 별로 Wide & Deep 데이터 생성
        # X_wide_train = wide_preprocessor.fit_transform(fold_train)
        # X_wide_valid = wide_preprocessor.transform(fold_valid)
        # X_wide_test = wide_preprocessor.transform(fold_test)
        X_tab_train = tab_preprocessor.fit_transform(fold_train)
        X_tab_valid = tab_preprocessor.transform(fold_valid)
        X_tab_test = tab_preprocessor.transform(fold_test)

        # Target 값: 정수형 (0,1)
        y_train = fold_train['임신 성공 여부'].astype(int).values
        y_valid = fold_valid['임신 성공 여부'].astype(int).values

        
        # BayesianTabMlp 모델 구성
        model = BayesianTabMlp(
            column_idx=tab_preprocessor.column_idx,  
            # column_idx (Dict[str, int]): TabMlp 모델에 입력될 피처들의 인덱스를 담은 딕셔너리.
            # 예시: {'education': 0, 'relationship': 1, 'workclass': 2, ...}
            
            cat_embed_input=tab_preprocessor.cat_embed_input,  
            # cat_embed_input (Optional[List[Tuple[str, int, int]]]): 
            # 범주형 피처에 대해 (컬럼명, 고유값 개수, 임베딩 차원) 튜플로 구성된 리스트.
            # 기본값: None (제공하지 않을 경우 임베딩이 적용되지 않음)
            
            continuous_cols=continuous_cols,  
            # continuous_cols (Optional[List[str]]): 연속형(수치형) 피처의 이름 리스트.
            # 기본값: None (연속형 피처가 없을 경우)
            
            mlp_hidden_dims=[200, 100],
            # mlp_hidden_dims (List[int]): MLP의 각 Dense layer에 사용될 뉴런 수를 정의하는 리스트.
            # 기본값: [200, 100]이지만, 여기서는 [128, 64]를 사용.
            
            prior_sigma_1=1.0,
            # prior_sigma_1 (float): 첫 번째 Gaussian 분포의 sigma (표준편차)에 대한 prior 값.
            # 기본값: 1.0
            # 후보 옵션: 데이터 특성에 따라 1.0, 0.5 등으로 조정 가능.
            
            prior_sigma_2=0.002,
            # prior_sigma_2 (float): 두 번째 Gaussian 분포의 sigma에 대한 prior 값.
            # 기본값: 0.002
            # 후보 옵션: 0.002, 0.001 등으로 변경 가능.
            
            prior_pi=0.8,
            # prior_pi (float): 두 Gaussian 분포를 혼합할 때 사용하는 scaling factor.
            # 기본값: 0.8
            # 후보 옵션: 0.8, 0.5 등 문제에 따라 조정 가능.
            
            posterior_mu_init=0,
            # posterior_mu_init (float): 가중치의 posterior 분포의 평균 초기값.
            # 기본값: 0.0
            
            posterior_rho_init=-7.0,
            # posterior_rho_init (float): 가중치의 posterior 분포의 rho 초기값.
            # 기본값: -7.0
            
            pred_dim=1  
            # pred_dim (int): 출력 차원의 크기.
            # 이진 분류 문제에서는 1로 설정합니다.
        )


        
        
        # 옵티마이저 및 학습률 스케줄러 설정
        optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)  # Set your desired learning rate here
        
        
        # EarlyStopping 콜백 설정 (patience=5, min_delta=0.001)
        early_stopping = EarlyStopping(
            monitor="val_loss", 
            patience=5, 
            min_delta=0.001, 
            verbose=1
        )
        
        # 현재 날짜/시간을 포함한 파일 이름 생성
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        model_path = f"saved_models/best_model_{timestamp}.pt"
        
        # ✅ 2. ModelCheckpoint: 최상의 모델을 자동 저장
        model_checkpoint = ModelCheckpoint(
            filepath=model_path,  # 모델 저장 경로
            monitor="val_loss",        # 감시할 지표 ('val_loss' 또는 'val_acc')
            save_best_only=True        # 가장 좋은 성능의 모델만 저장
        )
        

        # BayesianTrainer 생성 (metrics는 Accuracy 클래스를 그대로 전달)
        trainer = BayesianTrainer(
            model=model,
            objective="binary",
            optimizer=optimizer,
            callbacks=[early_stopping, model_checkpoint],  # 조기 종료 (3 에포크 동안 개선 없으면 종료)
            metrics=[Accuracy],
            verbose=1,                  # 학습 로그 출력 (기본값: 1)
            seed=333                    # 랜덤 시드 설정 (기본값: 1)

        )

        

        # 내부 validation split 없이 fold_train만 사용하여 학습
        trainer.fit(
        X_tab=X_tab_train,       # 필수: 학습용 tabular 데이터 (ndarray)
        target=y_train,          # 필수: 학습 타겟 값 (ndarray)
        X_tab_val=X_tab_valid,   # 선택: 검증 데이터 (ndarray). 직접 전달하지 않을 경우, val_split 사용 가능
        target_val=y_valid,      # 선택: 검증 타겟 값 (ndarray)
        # val_split=0,         # X_tab_val과 target_val 대신 학습/검증 분할 비율 지정 가능 (예: 20% 검증 데이터)
        n_epochs=100,            # 에포크 수 (기본값: 1, 여기서는 100 에포크)
        batch_size=1024,          # 배치 크기 (기본값: 32, 여기서는 128)
        )
        
        
        
        # 모델 학습 후 Validation 예측 코드:
        # 이미 확률이 계산되어 있는 컬럼을 사용합니다.
        valid_probs = trainer.predict_proba(X_tab=X_tab_valid)[:,1] ####
        valid_pred = (valid_probs > 0.5).astype(int)

        

        # 실제 정답 
        y_valid = fold_valid['임신 성공 여부'].values.astype(int)


        
        # 평가 지표 계산: 클래스 1의 확률 사용
        fold_auc = roc_auc_score(y_valid, valid_probs)
        print(f"Seed[{seed:<3}] Fold {fold + 1} | AUC: {fold_auc:.6f}")
        
        auc_scores.append(fold_auc)
        
        total_auc.append(fold_auc)
        # Test 데이터 예측 (각 fold의 모델로 예측한 결과 저장)
        test_pred = trainer.predict_proba(X_tab=X_tab_test)[:,1]
        test_preds.append(test_pred)
    
    # Fold 별 평균 성능 출력
    avg_auc = np.mean(auc_scores)
    
    print("-" * 80)
    print(f"Seed[{seed:<3}] Average Metrics | AUC: {avg_auc:.7f}")
    print("-" * 80)

# 전체 Validation 평균 성능 출력
val_auc = np.mean(total_auc)
print("-" * 80)
print(f"Validation Average Metrics | AUC: {val_auc:.7f}")

finish_time = time.time()
total_time = finish_time - start_time 

print(total_time)

C:\Users\User\AppData\Local\Temp\ipykernel_5824\3576064510.py:69: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_train['배란 유도 유형'] = df_train['배란 유도 유형'].replace(mapping)
C:\Users\User\AppData\Local\Temp\ipykernel_5824\3576064510.py:70: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_test['배란 유도 유형'] = df_test['배란 유도 유형'].replace(mapping)
C:\Users\User\AppData\Local\Temp\ipykernel_5824\3576064510.py:199: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.Cate

KeyboardInterrupt: 

In [ ]:
# stratify_label = train["임신 성공 여부"].astype(str)
old_auc = 0.7289820 * 100

new_auc = val_auc * 100


def calculate_change(old_value, new_value):
    change = new_value - old_value
    percentage_change = (change / old_value) * 100 if old_value != 0 else float('inf')
    return change, percentage_change

def format_change(change):
    return f"{change:+.6f}"

# 각 지표의 변화량 계산
auc_change, auc_pct = calculate_change(old_auc, new_auc)


# 결과 출력
print("\n====== 모델 성능 변화 ======")
print(f"{'Metric':<8}  {'AUC':>12}")
print("-" * 30)
print(f"{'Old':<8}  {old_auc:>12.6f}")
print(f"{'New':<8}  {new_auc:>12.6f}")
print(f"{'Change':<8}  {format_change(auc_change):>12}")
print(f"{'% Change':<8}  {auc_pct:>11.4f}%")
print("=" * 30)

In [ ]:
tmp_submission = pd.DataFrame({f'tabm_{data_seed}': np.mean(test_preds, axis=0)})
tmp_submission

In [ ]:
from LG_Aimers_6th.cal_auc import calculate_auc
score = calculate_auc(tmp_submission, seed=data_seed)
print(f'[seed {data_seed}]: {score}')

- 기본 test 데이터 AUC : 0.732737

## Submission

In [ ]:
sample_submission = pd.read_csv(sample_path)
# test_preds
# sample_submission['임신 성공 확률'] = np.mean(test_preds, axis=0)

# ratio = train['임신 성공 여부'].value_counts(normalize=True)[1]
# real_true_count = int(ratio * len(sample_submission))
# print(f'test의 True 갯수: {real_true_count:<5} (추정)')

# count = (sample_submission['임신 성공 확률'] >= 0.5).sum()
# print(f'test의 True 갯수: {count:<5} (예측 결과)')




In [ ]:
now = datetime.datetime.now().strftime("%m%d_%H%M")

In [ ]:
submission_path = 'Submission'
if not os.path.exists(submission_path):
    os.makedirs(submission_path)

code_dir = 'Code'
if not os.path.exists(code_dir):
    os.makedirs(code_dir)


submission_name = f"submission_{now}.csv"
new_notebook_name = f"code_{now}.ipynb"

sample_submission.to_csv(os.path.join(submission_path, submission_name), index=False)


# 현재 노트북 파일 경로 직접 지정 (실제 노트북 파일명으로 수정)
current_notebook = os.path.join(os.getcwd(), "WideDeep_TabMlp_임신 여부.ipynb")

new_notebook_path = os.path.join(code_dir, new_notebook_name)

# 노트북 파일 복사
shutil.copy(current_notebook, new_notebook_path)

print(f"Notebook saved in '{code_dir}' as '{new_notebook_name}'")


In [ ]:
# 📌 SQLite 데이터베이스 설정
db_path = "experiment_results.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# 📌 테이블 생성 (처음 실행 시)
cursor.execute('''
CREATE TABLE IF NOT EXISTS experiments (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    code_name TEXT,
    experiment_desc TEXT,
    auc REAL,
    acc REAL,
    f1 REAL
)
''')

In [ ]:
# 데이터 삽입
cursor.execute('''
INSERT INTO experiments (code_name, experiment_desc, auc, acc, f1)
VALUES (?, ?, ?, ?, ?)
''', (new_notebook_name, experiment_desc.strip(), new_auc, new_acc, new_f1))

# 변경사항 저장 & 연결 종료
conn.commit()
conn.close()

print(f"Experiment '{new_notebook_name}' successfully saved in database")

In [ ]:
import sqlite3
import pandas as pd

# SQLite 데이터 조회 함수
def get_experiment_results(db_path="experiment_results.db", num_results=10):
    """
    SQLite 데이터베이스에서 중복된 실험 데이터를 제거하고, 최근 num_results개의 실험 데이터를 불러오는 함수.
    Returns:
        - Pandas DataFrame: 중복 제거된 실험 데이터
    """
    conn = sqlite3.connect(db_path)

    # 중복 제거 & 최신 데이터 선택하는 SQL 쿼리
    query = f"""
    SELECT * FROM experiments
    WHERE id IN (
        SELECT MAX(id)  -- 가장 최신 데이터 선택
        FROM experiments
        GROUP BY code_name -- id 제외하고 중복 판단
    )
    ORDER BY id DESC  -- 최신 데이터부터 정렬
    LIMIT {num_results};
    """

    df = pd.read_sql_query(query, conn)
    conn.close()

    return df

In [ ]:
df_results = get_experiment_results(num_results=100)
df_results.to_csv('experiment_results.csv', index=False, encoding='utf-8-sig', float_format='%.4f')